# Data Processing of Test Data

Find missing files

In [15]:
# Existing imports
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import os
import json
import random
import openai
import anthropic

# Load environment variables
load_dotenv()

# Initialize API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

if not OPENAI_API_KEY or not ANTHROPIC_API_KEY:
    raise ValueError(
        "API keys for OpenAI and Anthropic must be set in the .env file.")

# Define paths using Pathlib
path_groundTruth_excel = Path(
    '../test-contracts/dataset/groundTruth/groundTruth.xlsx')
path_hex_folder = Path('../test-contracts/dataset/groundTruth/hex')
path_sol_folder = Path('../test-contracts/dataset/groundTruth/sol')

# Read the Excel file
df_groundTruth = pd.read_excel(path_groundTruth_excel, engine='openpyxl')

list_groundTruth_address = df_groundTruth['address'].tolist()
print('Number of groundTruth addresses: ', len(list_groundTruth_address))

# List filenames without extensions using Pathlib
list_filenames_hex = [f.stem for f in path_hex_folder.iterdir() if f.is_file()]
list_filenames_sol = [f.stem for f in path_sol_folder.iterdir() if f.is_file()]


def get_missing_addresses(
    ground_truth_addresses: list,
    existing_hex_files: list,
    existing_sol_files: list
) -> tuple:
    """
    Identify missing hex and sol files based on ground truth addresses.

    Args:
        ground_truth_addresses (list): List of ground truth contract addresses.
        existing_hex_files (list): List of existing hex file names (without extensions).
        existing_sol_files (list): List of existing sol file names (without extensions).

    Returns:
        tuple: Two lists containing missing hex addresses and missing sol addresses respectively.
    """
    # Convert lists to sets for faster lookup and ensure case-insensitivity
    ground_truth_set = {addr.strip().lower()
                        for addr in ground_truth_addresses}
    existing_hex_set = {fname.strip().lower() for fname in existing_hex_files}
    existing_sol_set = {fname.strip().lower() for fname in existing_sol_files}

    # Find missing hex and sol addresses using set operations
    missing_hex_files = list(ground_truth_set - existing_hex_set)
    missing_sol_files = list(ground_truth_set - existing_sol_set)

    # Log the missing counts
    print(f"Number of missing hex files: {len(missing_hex_files)}")
    print(f"Number of missing sol files: {len(missing_sol_files)}")

    return missing_hex_files, missing_sol_files

# Find missing hex and sol files using the improved function
missing_hex, missing_sol = get_missing_addresses(
    list_groundTruth_address,
    list_filenames_hex,
    list_filenames_sol
)

print(f"Missing Addresses are: \n hex: {missing_hex}, \n sol: {missing_sol}")
print('The number of missing addresses: hex: ',
      len(missing_hex), '; sol: ', len(missing_sol))

Number of groundTruth addresses:  68
Number of missing hex files: 1
Number of missing sol files: 1
Missing Addresses are: 
 hex: ['0xcb6cd204d783dc8d66896a6def5867d332228d7b'], 
 sol: ['0xcb6cd204d783dc8d66896a6def5867d332228d7b']
The number of missing addresses: hex:  1 ; sol:  1


# Experiment

## Setup

In [16]:
# Functions for Setup

def load_prompts():
    """Load prompts from the prompts directory."""
    prompts = {}
    prompt_dir = Path('prompts/')
    basic_prompt_path = prompt_dir / 'basic_ai_prompt.txt'
    developer_meta_prompt_path = prompt_dir / 'developer_meta_prompt.txt'

    with open(basic_prompt_path, 'r') as file:
        prompts['basic'] = file.read()

    with open(developer_meta_prompt_path, 'r') as file:
        prompts['developer_meta'] = file.read()

    return prompts


def load_contracts(contract_dir='../test-contracts/dataset/groundTruth/'):
    """Load smart contracts from the specified directory."""
    contracts = []
    for file in os.listdir(contract_dir):
        if file.endswith('.sol'):
            with open(os.path.join(contract_dir, file), 'r') as f:
                contracts.append({'filename': file, 'code': f.read()})
    return contracts


def initialize_openai_client():
    """Initialize the OpenAI client."""
    return openai.ChatCompletion()


def initialize_anthropic_client():
    """Initialize the Anthropic client."""
    return anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)


def audit_contract_openai(model, prompt, contract_code):
    """Audit a single contract using OpenAI's GPT-4-O model."""
    client = initialize_openai_client()
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a smart contract auditor."},
                {"role": "user", "content": prompt.replace(
                    "[Insert smart contract code here]", contract_code)}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_anthropic(model, prompt, contract_code):
    """Audit a single contract using Anthropic's Claude 3.5 Sonnet model."""
    client = initialize_anthropic_client()
    try:
        response = client.completions.create(
            model=model,
            prompt=prompt.replace(
                "[Insert smart contract code here]", contract_code),
            max_tokens=3000,
            temperature=0.7
        )
        return response.completion
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_openai(model, prompt, contract_code):
    """Audit a single contract using OpenAI's GPT-4-O model."""
    client = initialize_openai_client()
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a smart contract auditor."},
                {"role": "user", "content": prompt.replace(
                    "[Insert smart contract code here]", contract_code)}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_anthropic(model, prompt, contract_code):
    """Audit a single contract using Anthropic's Claude 3.5 Sonnet model."""
    client = initialize_anthropic_client()
    try:
        response = client.completions.create(
            model=model,
            prompt=prompt.replace(
                "[Insert smart contract code here]", contract_code),
            max_tokens=3000,
            temperature=0.7
        )
        return response.completion
    except Exception as e:
        return f"Error: {str(e)}"
    

def evaluate_audit(audit_result, known_errors):
    """
    Evaluate whether the audit_result contains the known_errors.
    
    Parameters:
    - audit_result (str): The text output from the audit model.
    - known_errors (list): A list of known error descriptions.
    
    Returns:
    - bool: True if all known errors are found, False otherwise.
    """
    audit_result_lower = audit_result.lower()
    return all(error.lower() in audit_result_lower for error in known_errors)


def analyze_results(results, total_contracts=20):
    """Analyze the audit results."""
    analysis = {}
    for model_name in results:
        analysis[model_name] = {}
        for prompt_type in results[model_name]:
            successes = sum(
                1 for result in results[model_name][prompt_type] if result['found_errors'])
            failure = total_contracts - successes
            analysis[model_name][prompt_type] = {
                'Success': successes,
                'Failure': failure,
                'Success Rate (%)': round((successes / total_contracts) * 100, 2)
            }
    return analysis

## Performing the audit

In [13]:
# Load prompts and contracts
prompts = load_prompts()
contracts = load_contracts()

# Define models
models = {
    'GPT4O': 'gpt-4o',
    'Claude3.5_Sonnet': 'claude-3-5-sonnet-20240620'
}

# Define prompt types
prompt_types = ['basic', 'developer_meta']

# Initialize results storage
results = {
    'GPT4O': {'basic': [], 'developer_meta': []},
    'Claude3.5_Sonnet': {'basic': [], 'developer_meta': []}
}

# Iterate over each contract
for contract in contracts[:20]:  # Ensure only 20 contracts are processed
    filename = contract['filename']
    code = contract['code']
    print(filename)
    break

    # Retrieve known errors for this contract
    # Assuming df_groundTruth has 'address' and 'errors' columns
    # Adjust the column names based on your actual DataFrame
    known_errors = df_groundTruth[df_groundTruth['address']
                                  == filename]['errors'].tolist()

    for model_name, model_identifier in models.items():
        for prompt_type in prompt_types:
            if model_name == 'GPT4O':
                audit_result = audit_contract_openai(
                    model_identifier, prompts[prompt_type], code)
            elif model_name == 'Claude3.5_Sonnet':
                audit_result = audit_contract_anthropic(
                    model_identifier, prompts[prompt_type], code)
            else:
                audit_result = "Unsupported model."

            # Evaluate the audit result
            found_errors = evaluate_audit(audit_result, known_errors)

            # Store the result
            results[model_name][prompt_type].append({
                'contract': filename,
                'found_errors': found_errors
            })

            print(
                f"Completed {model_name} with {prompt_type} on {filename}: {'Success' if found_errors else 'Failure'}")

FileNotFoundError: [Errno 2] No such file or directory: '../test-contracts/RugPull/dataset/groundTruth/'

### Saving the results to CSV

In [ ]:
# Create audit_reports directory if it doesn't exist
os.makedirs('audit_reports', exist_ok=True)

# Save the results to CSV files
for model_name in results:
    for prompt_type in results[model_name]:
        df_results = pd.DataFrame(results[model_name][prompt_type])
        csv_path = f'audit_reports/{model_name.lower()}_{prompt_type}_results.csv'
        df_results.to_csv(csv_path, index=False)
        print(f"Results saved to {csv_path}")

### Analyzing the results

In [ ]:
def analyze_results(results, total_contracts=20):
    """Analyze the audit results."""
    analysis = {}
    for model_name in results:
        analysis[model_name] = {}
        for prompt_type in results[model_name]:
            successes = sum(
                1 for result in results[model_name][prompt_type] if result['found_errors'])
            failure = total_contracts - successes
            analysis[model_name][prompt_type] = {
                'Success': successes,
                'Failure': failure,
                'Success Rate (%)': round((successes / total_contracts) * 100, 2)
            }
    return analysis


# Perform analysis
analysis = analyze_results(results)

# Display the analysis
for model, prompt_data in analysis.items():
    print(f"\nModel: {model}")
    for prompt, data in prompt_data.items():
        print(f"  Prompt: {prompt}")
        for key, value in data.items():
            print(f"    {key}: {value}")